### Project description

This project is about visualizing insights from expenses data provided from Privatbank bank statements.

Goals: 
- predict missing categories and clean up the data
- understand the most frequent expense
- find out the most costly category
- find out the weekday on which most money is spend
- find out the hour, or time range on which most money is spend


### Project plan
- fix issue with ukrainian language
- clean up the data
- predict missing categories by looking at the transaction description
- find out the most costly category
- find out the place with the biggest amount of transactions
- find out the most costly category
- find out the weekday on which most money is spend
- find out the hour, or time range on which most money is spend
- plot correlation between time and amount of money spent
- other plots? 

In [45]:
import pandas as pd

In [46]:
df = pd.read_csv("statements.csv", encoding="utf_8")

In [47]:
df.head(10)

,Дата,Час,Категорія,Картка,Опис операції,Сума у валюті картки,Валюта картки,Сума у валюті транзакції,Валюта транзакції,Залишок на кінець періоду,Валюта залишку
0,04.12.2018,18:47,Туризм,Картка Універсальна (****2297),Послуги туризму та транспортних перевезень: Pr...,-2.50,грн,2.50,грн,762.17,грн
1,04.12.2018,18:34,Інше,Картка Універсальна (****2297),"Депозит через POS-терминал, ПЕРЕВОД СРЕДСТВ: H...",634.00,грн,634.00,грн,764.67,грн
2,04.12.2018,14:33,Інше,Картка Універсальна (****2297),"Продукти: Сильпо, Львiв, вул. Пiд Дубом, 7",-63.59,грн,63.59,грн,130.67,грн
3,04.12.2018,13:15,Інше,Картка Універсальна (****2297),Оплата в інтернет-магазині,-265.00,грн,265.00,грн,194.26,грн
4,04.12.2018,00:54,Розваги,Картка Універсальна (****2297),"Музика: ITUNES.COM/BILL, ITUNES.COM",-70.54,грн,2.49,дол,459.26,грн
5,03.12.2018,19:27,Інше,Картка Універсальна (****2297),Переказ з карти ПриватБанку через додаток Прив...,393.00,грн,393.00,грн,529.80,грн
6,03.12.2018,19:10,Туризм,Картка Універсальна (****2297),"Залізничні послуги: WWW.UZ.GOV.UA, KYIV",-1181.40,грн,1181.40,грн,136.80,грн
7,03.12.2018,19:01,Інше,Картка Універсальна (****2297),Переказ зі своєї карти 51**85 через додаток Пр...,1000.00,грн,1000.00,грн,1318.20,грн
8,03.12.2018,18:57,Продукти харчування,Картка Універсальна (****2297),"Продукти: FOP SAVARYN A.I., LVIV",-45.60,грн,45.60,грн,318.20,грн
9,03.12.2018,15:50,"Кафе, бари, ресторани",Картка Універсальна (****2297),"Ресторан: TRAPEZNA UCU, LVIV",-27.25,грн,27.25,грн,363.80,грн


In [48]:
df.dtypes

Дата                          object
Час                           object
Категорія                     object
Картка                        object
Опис операції                 object
Сума у валюті картки         float64
Валюта картки                 object
Сума у валюті транзакції     float64
Валюта транзакції             object
Залишок на кінець періоду    float64
Валюта залишку                object
dtype: object

In [49]:
df = df.drop(columns=["Валюта картки",
                        "Валюта залишку", "Картка"])

KeyError: "['\\xd0\\x92\\xd0\\xb0\\xd0\\xbb\\xd1\\x8e\\xd1\\x82\\xd0\\xb0 \\xd0\\xba\\xd0\\xb0\\xd1\\x80\\xd1\\x82\\xd0\\xba\\xd0\\xb8'\n '\\xd0\\x92\\xd0\\xb0\\xd0\\xbb\\xd1\\x8e\\xd1\\x82\\xd0\\xb0 \\xd0\\xb7\\xd0\\xb0\\xd0\\xbb\\xd0\\xb8\\xd1\\x88\\xd0\\xba\\xd1\\x83'\n '\\xd0\\x9a\\xd0\\xb0\\xd1\\x80\\xd1\\x82\\xd0\\xba\\xd0\\xb0'] not found in axis"

df1

In [50]:
df.head(40)

,Дата,Час,Категорія,Картка,Опис операції,Сума у валюті картки,Валюта картки,Сума у валюті транзакції,Валюта транзакції,Залишок на кінець періоду,Валюта залишку
0,04.12.2018,18:47,Туризм,Картка Універсальна (****2297),Послуги туризму та транспортних перевезень: Pr...,-2.50,грн,2.50,грн,762.17,грн
1,04.12.2018,18:34,Інше,Картка Універсальна (****2297),"Депозит через POS-терминал, ПЕРЕВОД СРЕДСТВ: H...",634.00,грн,634.00,грн,764.67,грн
2,04.12.2018,14:33,Інше,Картка Універсальна (****2297),"Продукти: Сильпо, Львiв, вул. Пiд Дубом, 7",-63.59,грн,63.59,грн,130.67,грн
3,04.12.2018,13:15,Інше,Картка Універсальна (****2297),Оплата в інтернет-магазині,-265.00,грн,265.00,грн,194.26,грн
4,04.12.2018,00:54,Розваги,Картка Універсальна (****2297),"Музика: ITUNES.COM/BILL, ITUNES.COM",-70.54,грн,2.49,дол,459.26,грн
5,03.12.2018,19:27,Інше,Картка Універсальна (****2297),Переказ з карти ПриватБанку через додаток Прив...,393.00,грн,393.00,грн,529.80,грн
6,03.12.2018,19:10,Туризм,Картка Універсальна (****2297),"Залізничні послуги: WWW.UZ.GOV.UA, KYIV",-1181.40,грн,1181.40,грн,136.80,грн
7,03.12.2018,19:01,Інше,Картка Універсальна (****2297),Переказ зі своєї карти 51**85 через додаток Пр...,1000.00,грн,1000.00,грн,1318.20,грн
8,03.12.2018,18:57,Продукти харчування,Картка Універсальна (****2297),"Продукти: FOP SAVARYN A.I., LVIV",-45.60,грн,45.60,грн,318.20,грн
9,03.12.2018,15:50,"Кафе, бари, ресторани",Картка Універсальна (****2297),"Ресторан: TRAPEZNA UCU, LVIV",-27.25,грн,27.25,грн,363.80,грн


### Working with keywords

In [26]:
df = df.replace({'Туризм': 'tourism',
 'Продукти': 'food',
 'Музика': 'music',
 'Сильпо': 'silpo',
 'Одяг': 'clothes',
 'ЛИМУЗИНЫ': 'taxi',
 'UKLON': 'taxi',
 'Кава': 'coffee',
 'Послуги туризму та транспортних перевезень': 'tram',
 'KINOMARKET': 'cinema'})

In [28]:
df.head(10)

,Дата,Час,Категорія,Опис операції,Сума у валюті картки,Сума у валюті транзакції,Валюта транзакції,Залишок на кінець періоду
0,04.12.2018,18:47,tourism,Послуги туризму та транспортних перевезень: Pr...,-2.50,2.50,грн,762.17
1,04.12.2018,18:34,Інше,"Депозит через POS-терминал, ПЕРЕВОД СРЕДСТВ: H...",634.00,634.00,грн,764.67
2,04.12.2018,14:33,Інше,"Продукти: Сильпо, Львiв, вул. Пiд Дубом, 7",-63.59,63.59,грн,130.67
3,04.12.2018,13:15,Інше,Оплата в інтернет-магазині,-265.00,265.00,грн,194.26
4,04.12.2018,00:54,Розваги,"Музика: ITUNES.COM/BILL, ITUNES.COM",-70.54,2.49,дол,459.26
5,03.12.2018,19:27,Інше,Переказ з карти ПриватБанку через додаток Прив...,393.00,393.00,грн,529.80
6,03.12.2018,19:10,tourism,"Залізничні послуги: WWW.UZ.GOV.UA, KYIV",-1181.40,1181.40,грн,136.80
7,03.12.2018,19:01,Інше,Переказ зі своєї карти 51**85 через додаток Пр...,1000.00,1000.00,грн,1318.20
8,03.12.2018,18:57,Продукти харчування,"Продукти: FOP SAVARYN A.I., LVIV",-45.60,45.60,грн,318.20
9,03.12.2018,15:50,"Кафе, бари, ресторани","Ресторан: TRAPEZNA UCU, LVIV",-27.25,27.25,грн,363.80


### Renaming columns

In [59]:
df.rename(columns={'Дата': 'date',
                     'Час':'time',
                     'Категорія':'category',
                     'Опис операції':'description',
                     'Сума у валюті картки':'transaction',
                     'Сума у валюті транзакції':'abs',
                     'Валюта транзакції':'currency',
                     'Залишок на кінець періоду':'balance',
                    }, 
                 inplace=True)

In [60]:
df.head(10)

,Дата,Час,Категорія,Картка,Опис операції,Сума у валюті картки,Валюта картки,Сума у валюті транзакції,Валюта транзакції,Залишок на кінець періоду,Валюта залишку
0,04.12.2018,18:47,Туризм,Картка Універсальна (****2297),Послуги туризму та транспортних перевезень: Pr...,-2.50,грн,2.50,грн,762.17,грн
1,04.12.2018,18:34,Інше,Картка Універсальна (****2297),"Депозит через POS-терминал, ПЕРЕВОД СРЕДСТВ: H...",634.00,грн,634.00,грн,764.67,грн
2,04.12.2018,14:33,Інше,Картка Універсальна (****2297),"Продукти: Сильпо, Львiв, вул. Пiд Дубом, 7",-63.59,грн,63.59,грн,130.67,грн
3,04.12.2018,13:15,Інше,Картка Універсальна (****2297),Оплата в інтернет-магазині,-265.00,грн,265.00,грн,194.26,грн
4,04.12.2018,00:54,Розваги,Картка Універсальна (****2297),"Музика: ITUNES.COM/BILL, ITUNES.COM",-70.54,грн,2.49,дол,459.26,грн
5,03.12.2018,19:27,Інше,Картка Універсальна (****2297),Переказ з карти ПриватБанку через додаток Прив...,393.00,грн,393.00,грн,529.80,грн
6,03.12.2018,19:10,Туризм,Картка Універсальна (****2297),"Залізничні послуги: WWW.UZ.GOV.UA, KYIV",-1181.40,грн,1181.40,грн,136.80,грн
7,03.12.2018,19:01,Інше,Картка Універсальна (****2297),Переказ зі своєї карти 51**85 через додаток Пр...,1000.00,грн,1000.00,грн,1318.20,грн
8,03.12.2018,18:57,Продукти харчування,Картка Універсальна (****2297),"Продукти: FOP SAVARYN A.I., LVIV",-45.60,грн,45.60,грн,318.20,грн
9,03.12.2018,15:50,"Кафе, бари, ресторани",Картка Універсальна (****2297),"Ресторан: TRAPEZNA UCU, LVIV",-27.25,грн,27.25,грн,363.80,грн


In [54]:
df['description'] = df['description'].replace({'Туризм': 'tourism',
 'Продукти': 'food',
 'Музика': 'music',
 'Сильпо': 'silpo',
 'Одяг': 'clothes',
 'ЛИМУЗИНЫ': 'taxi',
 'UKLON': 'taxi',
 'Кава': 'coffee',
 'Послуги туризму та транспортних перевезень': 'tram',
 'KINOMARKET': 'cinema'})

KeyError: 'description'

In [55]:
df.head(10)

,Дата,Час,Категорія,Картка,Опис операції,Сума у валюті картки,Валюта картки,Сума у валюті транзакції,Валюта транзакції,Залишок на кінець періоду,Валюта залишку
0,04.12.2018,18:47,Туризм,Картка Універсальна (****2297),Послуги туризму та транспортних перевезень: Pr...,-2.50,грн,2.50,грн,762.17,грн
1,04.12.2018,18:34,Інше,Картка Універсальна (****2297),"Депозит через POS-терминал, ПЕРЕВОД СРЕДСТВ: H...",634.00,грн,634.00,грн,764.67,грн
2,04.12.2018,14:33,Інше,Картка Універсальна (****2297),"Продукти: Сильпо, Львiв, вул. Пiд Дубом, 7",-63.59,грн,63.59,грн,130.67,грн
3,04.12.2018,13:15,Інше,Картка Універсальна (****2297),Оплата в інтернет-магазині,-265.00,грн,265.00,грн,194.26,грн
4,04.12.2018,00:54,Розваги,Картка Універсальна (****2297),"Музика: ITUNES.COM/BILL, ITUNES.COM",-70.54,грн,2.49,дол,459.26,грн
5,03.12.2018,19:27,Інше,Картка Універсальна (****2297),Переказ з карти ПриватБанку через додаток Прив...,393.00,грн,393.00,грн,529.80,грн
6,03.12.2018,19:10,Туризм,Картка Універсальна (****2297),"Залізничні послуги: WWW.UZ.GOV.UA, KYIV",-1181.40,грн,1181.40,грн,136.80,грн
7,03.12.2018,19:01,Інше,Картка Універсальна (****2297),Переказ зі своєї карти 51**85 через додаток Пр...,1000.00,грн,1000.00,грн,1318.20,грн
8,03.12.2018,18:57,Продукти харчування,Картка Універсальна (****2297),"Продукти: FOP SAVARYN A.I., LVIV",-45.60,грн,45.60,грн,318.20,грн
9,03.12.2018,15:50,"Кафе, бари, ресторани",Картка Універсальна (****2297),"Ресторан: TRAPEZNA UCU, LVIV",-27.25,грн,27.25,грн,363.80,грн


In [56]:
df.description

AttributeError: 'DataFrame' object has no attribute 'description'

In [57]:
df['description'].value_counts().idxmax()

KeyError: 'description'

In [58]:
df['category'].value_counts().idxmax().decode('utf8')

KeyError: 'category'